# Validate raw PSD

activate spikebias

In [ ]:
# import packages
import os
import numpy as np
import spikeinterface as si

# setup project path
proj_path = "/home/steeve/steeve/epfl/code/spikebias"

# setup paths
RAW_PATH_nv = os.path.join(proj_path, "dataset/00_raw/recording_marques_smith/")
RAW_PATH_ns = os.path.join(proj_path, "dataset/00_raw/recording_npx_spont/")
RAW_PATH_ne = os.path.join(proj_path, "dataset/00_raw/recording_npx_evoked/")
RAW_PATH_hv1 = os.path.join(proj_path, "dataset/00_raw/recording_horvath_probe1/")
RAW_PATH_hv2 = os.path.join(proj_path, "dataset/00_raw/recording_horvath_probe2/")
RAW_PATH_hv3 = os.path.join(proj_path, "dataset/00_raw/recording_horvath_probe3/")
RAW_PATH_hs1 = os.path.join(proj_path, "dataset/00_raw/recording_dense_probe1/")
RAW_PATH_hs2 = os.path.join(proj_path, "dataset/00_raw/recording_dense_probe2/")
RAW_PATH_hs3 = os.path.join(proj_path, "dataset/00_raw/recording_dense_probe3/")

## Load datasets

In [25]:
# load neuropixels traces
RawNV = si.load_extractor(RAW_PATH_nv)
RawNS = si.load_extractor(RAW_PATH_ns)
RawNE = si.load_extractor(RAW_PATH_ne)

# Load dense probe traces (depth 1,2,3)
# horvath data
RawHV1 = si.load_extractor(RAW_PATH_hv1)
RawHV2 = si.load_extractor(RAW_PATH_hv2)
RawHV3 = si.load_extractor(RAW_PATH_hv3)

# biophysical simulation
RawHS1 = si.load_extractor(RAW_PATH_hs1)
RawHS2 = si.load_extractor(RAW_PATH_hs2)
RawHS3 = si.load_extractor(RAW_PATH_hs3)

# 4 - get good sites (in cortex) **********************
# neuropixels
lyrs = ["L1", "L2_3", "L4", "L5", "L6"]
# silico
site_ly_ns = RawNS.get_property("layers")
site_ly_ns[site_ly_ns == "L2"] = "L2_3"
site_ly_ns[site_ly_ns == "L3"] = "L2_3"
sites_ns = np.where(np.isin(site_ly_ns, lyrs))[0]
# evoked
site_ly_ne = RawNE.get_property("layers")
site_ly_ne[site_ly_ne == "L2"] = "L2_3"
site_ly_ne[site_ly_ne == "L3"] = "L2_3"
sites_ne = np.where(np.isin(site_ly_ne, lyrs))[0]
# vivo
site_ly_nv = RawNV.get_property("layers")
sites_nv = np.where(np.isin(site_ly_nv, lyrs))[0]

# horvath
# silico
# probe 1 (select L1, L2/3)
site_ly_hs1 = RawHS1.get_property("layers")
site_ly_hs1 = np.array(["L2_3" if x == "L2" or x == "L3" else x for x in site_ly_hs1])
sites_hs1 = np.where(np.isin(site_ly_hs1, ["L1", "L2_3"]))[0]

# probe 2 (select L4 and L5)
site_ly_hs2 = RawHS2.get_property("layers")
sites_hs2 = np.where(np.isin(site_ly_hs2, ["L4", "L5"]))[0]

# probe 3 (select L6)
site_ly_hs3 = RawHS3.get_property("layers")
site_ly_hs3 = np.array(["L2_3" if x == "L2" or x == "L3" else x for x in site_ly_hs3])
sites_hs3 = np.where(np.isin(site_ly_hs3, ["L6"]))[0]

# vivo
# probe 1
site_ly_hv1 = RawHV1.get_property("layers")
sites_hv1 = np.where(np.isin(site_ly_hv1, ["L1", "L2_3"]))[0]
# probe 2
site_ly_hv2 = RawHV2.get_property("layers")
sites_hv2 = np.where(np.isin(site_ly_hv2, ["L4", "L5"]))[0]
# probe 3
site_ly_hv3 = RawHV3.get_property("layers")
sites_hv3 = np.where(np.isin(site_ly_hv3, ["L6"]))[0]

/home/steeve/steeve/epfl/code/spikebias/envs/spikebias/lib/python3.9/site-packages/spikeinterface/core/base.py:1079: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.101.2 is recommended
  warnings.warn(


In [ ]:
# horvath vivo (all layers)
# L1 (probe 1)
psd_pre_hv1_L1 = copy.copy(psd_pre_hv1)
psd_pre_hv1_L1["power"] = psd_pre_hv1["power"][site_ly_hv1[sites_hv1] == "L1", :]

# L23 (probe 1)
psd_pre_hv1_L23 = copy.copy(psd_pre_hv1)
psd_pre_hv1_L23["power"] = psd_pre_hv1["power"][site_ly_hv1[sites_hv1] == "L2_3", :]

# L4 (probe 2)
psd_pre_hv2_L4 = copy.copy(psd_pre_hv2)
psd_pre_hv2_L4["power"] = psd_pre_hv2["power"][site_ly_hv2[sites_hv2] == "L4", :]

# L5 (probe 2)
psd_pre_hv2_L5 = copy.copy(psd_pre_hv2)
psd_pre_hv2_L5["power"] = psd_pre_hv2["power"][site_ly_hv2[sites_hv2] == "L5", :]
# L6 (probe 3)
psd_pre_hv3_L6 = copy.copy(psd_pre_hv3)
psd_pre_hv3_L6["power"] = psd_pre_hv3["power"][site_ly_hv3[sites_hv3] == "L6", :]

# sites in target layers in each probe
psd_all_hv = dict()
psd_all_hv["power"] = np.vstack(
    [
        psd_pre_hv1_L1["power"],
        psd_pre_hv1_L23["power"],
        psd_pre_hv2_L4["power"],
        psd_pre_hv2_L5["power"],
        psd_pre_hv3_L6["power"],
    ]
)
psd_all_hv["freq"] = psd_pre_hv1["freq"]


# horvath biophy. spont. (all layers) **************************
# L1 (probe 1)
psd_pre_hs1_L1 = copy.copy(psd_pre_hs1)
psd_pre_hs1_L1["power"] = psd_pre_hs1["power"][site_ly_hs1[sites_hs1] == "L1", :]
# L23 (probe 1)
psd_pre_hs1_L23 = copy.copy(psd_pre_hs1)
psd_pre_hs1_L23["power"] = psd_pre_hs1["power"][site_ly_hs1[sites_hs1] == "L2_3", :]
# L4 (probe 2)
psd_pre_hs2_L4 = copy.copy(psd_pre_hs2)
psd_pre_hs2_L4["power"] = psd_pre_hs2["power"][site_ly_hs2[sites_hs2] == "L4", :]
# L5 (probe 2)
psd_pre_hs2_L5 = copy.copy(psd_pre_hs2)
psd_pre_hs2_L5["power"] = psd_pre_hs2["power"][site_ly_hs2[sites_hs2] == "L5", :]
# L6 (probe 3)
psd_pre_hs3_L6 = copy.copy(psd_pre_hs3)
psd_pre_hs3_L6["power"] = psd_pre_hs3["power"][site_ly_hs3[sites_hs3] == "L6", :]

# sites in target layers in each probe
psd_all_hs = dict()
psd_all_hs["power"] = np.vstack(
    [
        psd_pre_hs1_L1["power"],
        psd_pre_hs1_L23["power"],
        psd_pre_hs2_L4["power"],
        psd_pre_hs2_L5["power"],
        psd_pre_hs3_L6["power"],
    ]
)
psd_all_hs["freq"] = psd_pre_hs1["freq"]


FIG_SIZE = (1, 1)

# we rasterize, else dots increase file size by
# 6MB per plot
# takes 2 mins
YLIM_AX1 = [0, 0.02]
YLIM_AX2 = [0, 0.08]
N_MAJOR_TICKS = 4
ALPHA = 0.7
fig, ax1 = plt.subplots(1, 1, figsize=FIG_SIZE)

# Fit panel ************************************************************

ax1, alphas_lfp_hv, alphas_spiking_hv = power.plot_fits_all(
    ax1, psd_all_hv, prm["SFREQ_HV"], cl["COLOR_HV"], pm, pm_fit1, pm_fit2
)
ax1, alphas_lfp_nv, alphas_spiking_nv = power.plot_fits_all(
    ax1, psd_pre_nv, prm["SFREQ_NV"], cl["COLOR_NV"], pm, pm_fit1, pm_fit2
)
ax1, alphas_lfp_ns, alphas_spiking_ns = power.plot_fits_all(
    ax1, psd_pre_ns, prm["SFREQ_NS"], cl["COLOR_NS"], pm, pm_fit1, pm_fit2
)
ax1, alphas_lfp_ne, alphas_spiking_ne = power.plot_fits_all(
    ax1, psd_pre_ne, prm["SFREQ_NE"], cl["COLOR_NE"], pm, pm_fit1, pm_fit2
)
ax1, alphas_lfp_hs, alphas_spiking_hs = power.plot_fits_all(
    ax1, psd_all_hs, prm["SFREQ_HS"], cl["COLOR_HS"], pm, pm_fit1, pm_fit2
)

# axes legend
# esthetics
ax1.set_xscale("log")
ax1.set_yscale("log")
ax1.spines[["top", "right"]].set_visible(False)
# minor ticks
locmaj = matplotlib.ticker.LogLocator(base=10, numticks=N_MAJOR_TICKS)
locmin = matplotlib.ticker.LogLocator(
    base=10.0,
    subs=(0.2, 0.4, 0.6, 0.8),
    numticks=N_MINOR_TICKS,
)
ax1.tick_params(which="both")
ax1.xaxis.set_major_locator(locmaj)
ax1.xaxis.set_minor_locator(locmin)
ax1.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
# disconnect axes (R style)
ax1.spines["bottom"].set_position(("axes", -0.05))
ax1.yaxis.set_ticks_position("left")
ax1.spines["left"].set_position(("axes", -0.05))

# report slopes
# lfp band
print("\nLFP band")
print_slope(alphas_lfp_hv)
print_slope(alphas_lfp_nv)
print_slope(alphas_lfp_ns)
print_slope(alphas_lfp_ne)
print_slope(alphas_lfp_hs)

# spiking band
print("\nSpiking band")
print_slope(alphas_spiking_hv)
print_slope(alphas_spiking_nv)
print_slope(alphas_spiking_ns)
print_slope(alphas_spiking_ne)
print_slope(alphas_spiking_hs)

# print sites
print("\n")
print(f"""n={psd_all_hv["power"].shape[0]} sites""")
print(f"""n={psd_pre_nv["power"].shape[0]} sites""")
print(f"""n={psd_pre_ns["power"].shape[0]} sites""")
print(f"""n={psd_pre_ne["power"].shape[0]} sites""")
print(f"""n={psd_all_hs["power"].shape[0]} sites""")


# SNR **************************
print("\nPower SNR")

out_hv = power.get_power_snr(psd_all_hv["power"], psd_all_hv["freq"], 300, 6000, 90)
out_nv = power.get_power_snr(psd_pre_nv["power"], psd_pre_nv["freq"], 300, 6000, 90)
out_ns = power.get_power_snr(psd_pre_ns["power"], psd_pre_ns["freq"], 300, 6000, 90)
out_ne = power.get_power_snr(psd_pre_ne["power"], psd_pre_ne["freq"], 300, 6000, 90)
out_hs = power.get_power_snr(psd_all_hs["power"], psd_all_hs["freq"], 300, 6000, 90)
out_nb = power.get_power_snr(psd_pre_nb["power"], psd_pre_nb["freq"], 300, 6000, 90)

# save
if SAVE_FIG:
    plt.savefig(
        "figures/6_supp/fig2/fig2G_pre_power_fit_all_sites.svg",
        **savefig_cfg,
    )